# ECEn 360 Lab 2 - LTspice Simulations of a Transmission Line

## Part III

### Transmission line diagram

Create a simple SVG drawing and display it with ipython's display machinery.

In [1]:
from IPython.display import HTML, display

tline_drawing = '''
<svg height="100" width="500">

<defs>
<marker id="arrowhead" markerWidth="10" markerHeight="7" 
refX="0" refY="2" orient="auto">
  <polygon points="0 0, 10 2, 0 4" />
</marker>
</defs>
<line x1="30" y1="50" x2="70" y2="50" stroke="#000" stroke-width="2" marker-end="url(#arrowhead)" />

<g fill = "black">
    <text x = "10" y = "55" font-size = "14">
        <tspan>Z<tspan baseline-shift = "sub">in</tspan></tspan>
    </text>
    <text x = "200" y = "55" font-size = "14">
        <tspan>Z<tspan baseline-shift = "sub">0</tspan></tspan>
    </text>
</g>

<line x1="70" y1="25" x2="370" y2="25" stroke="#000" stroke-width="2" />
<line x1="70" y1="75" x2="370" y2="75" stroke="#000" stroke-width="2" />

<circle cx="70" cy="25" r="4" stroke="black" stroke-width="2" fill="white" />
<circle cx="70" cy="75" r="4" stroke="black" stroke-width="2" fill="white" />
<circle cx="370" cy="25" r="4" stroke="black" stroke-width="2" fill="white" />
<circle cx="370" cy="75" r="4" stroke="black" stroke-width="2" fill="white" />

</svg
'''
display(HTML(tline_drawing))

The characteristic impedance, $Z_0$, phase constant, $\beta$, and phase velocity, $u_p$, are

\begin{align}
    Z_0 &= \sqrt{\frac{L}{C}} \\
    \beta &= \omega \sqrt{L C} \\
    u_p &= \frac{1}{\sqrt{L C}}
\end{align}

where

$$\omega = 2 \pi f $$

To determine the overall capacitance we need to calculate the input impedance, $Z_{in}$ of the transmission line. The input impedance is

$$Z_{in} = Z_0 \frac{1 + \Gamma_{in}}{1 - \Gamma_{in}} $$

where

\begin{align}
    \Gamma_{in} &= \Gamma_L \exp(-j2\beta l)
\end{align}

since the generalized reflection coefficient is

\begin{align}
\Gamma(z) &= \Gamma_L \exp(j2\beta z)
\end{align}

and $\Gamma_{in} = \Gamma(z = -l)$. The load reflection coefficient is 

$$\Gamma_L = \frac{Z_L - Z_0}{Z_L + Z_0}$$

Since the load is an open circuit, $\Gamma_L = 1$. 

Note that $Z_{in}$ is in general complex so we can write it as

$$Z_{in} = R + jX$$

where the real part, $R$, is the resistive component of the impedance, and the imaginary part, $X$, is the reactance. Recall that the impedance for a capacitor is $Z_C = -j \frac{1}{\omega C} = jX$ where $X \lt 0$, while the impedance for an inductor is $Z_L = j \omega L = jX$ where $X \gt 0$. So, if the imaginary part of $Z_{in}$ is less than 0, it is capacitive, and if it is greater than 0 it is inductive.

Suppose when we calculate the input impedance it has no resistive component such that $Z_{in} = jX$, and that $X \lt 0$. The capacitance of the transmission line can be calculated as

$$C = \frac{-1}{\omega X}$$

### Now let's do the calculations

In [2]:
import math
from cmath import exp
import numpy as np

In [3]:
# Make sure we get what we expect for exp(jPi/2)
exp((math.pi/2)*1j)

(6.123233995736766e-17+1j)

In [4]:
# Function definitions
def beta(f, L, C):
    return 2 * math.pi * f * math.sqrt(L * C)

def gamma_in(f, L, C, l):
    b = beta(f, L, C)
    return exp(-2j*b*l)

def input_impedance(z0, gamma_in):
    return z0*(1 + gamma_in)/(1 - gamma_in)

# Parameters
L = 240e-9
C = 95e-12
frequencies = [0.5e6, 1e6, 2e6, 5e6, 10e6, 20e6]
l = 10.

# Calculations and display
z0 = math.sqrt(L/C)
up = 1/math.sqrt(L*C)
print('\nZ_0: {:6.4f} Ohms'.format(z0))
print('u_p: {:6.4e} m/s\n'.format(up))
print(' f (Hz)   beta (radians/m)    gamma_in         Z_in (Ohms)     Capacitance (F)')
print('------------------------------------------------------------------------------')
for f in frequencies:
    b = beta(f, L, C)
    g_in = gamma_in(f, L, C, l)
    z_in = input_impedance(z0, g_in)
    if np.imag(z_in) < 0:
        capacitance = -1/(2 * math.pi *f * np.imag(z_in))
        print('{:4.2e}      {:6.4f}        {:6.4f}   {:6.4f}    {:6.4e}'.format(f, b, g_in, z_in, capacitance))
    else:
        print('{:4.2e}      {:6.4f}        {:6.4f}   {:6.4f}   inductive'.format(f, b, g_in, z_in))


Z_0: 50.2625 Ohms
u_p: 2.0943e+08 m/s

 f (Hz)   beta (radians/m)    gamma_in         Z_in (Ohms)     Capacitance (F)
------------------------------------------------------------------------------
5.00e+05      0.0150        0.9553-0.2955j   0.0000-332.5460j    9.5719e-10
1.00e+06      0.0300        0.8253-0.5647j   0.0000-162.4745j    9.7957e-10
2.00e+06      0.0600        0.3623-0.9321j   0.0000-73.4628j    1.0832e-09
5.00e+06      0.1500        -0.9900-0.1409j   0.0000-3.5598j    8.9418e-09
1.00e+07      0.3000        0.9603+0.2791j   -0.0000+353.0584j   inductive
2.00e+07      0.6000        0.8442+0.5360j   -0.0000+172.9515j   inductive
